Create images & videos out of saved generation data

# Imports

In [ ]:
import sys
from pathlib import Path

import imageio
import torch
from IPython.display import HTML

sys.path.append(str(Path("..").resolve()))

from utils.misc import _normalize_elements_for_logging

# Experiment

In [ ]:
root_experiments_path = Path("/", "projects", "static2dynamic", "Thomas", "experiments")

In [ ]:
project_name = "GaussianProxy"
run_name = "video_time_encoder_trained"

In [ ]:
project_path = root_experiments_path / project_name
assert project_path.exists(), f"Project path {project_path} does not exist."

run_path = project_path / run_name
assert run_path.exists(), f"Run path {run_path} does not exist."

output_dir = Path(".", "artifacts_viz_outputs", project_name, run_name)
output_dir.mkdir(exist_ok=True, parents=True)
print("output dir:", output_dir)

# Show video of trajectories

In [ ]:
proc0_vids_path = run_path / "saved_artifacts" / "proc_0" / "trajectories"
assert proc0_vids_path.exists(), f"Proc0 vids path {proc0_vids_path} does not exist."
all_trajs = list(proc0_vids_path.rglob("*.pt"))
print(f"Found {len(all_trajs)} trajectories generated by process 0.")
last_traj_path = all_trajs[-1]
print(f"Last trajectory: {last_traj_path.name}")
last_traj = torch.load(last_traj_path).cpu()
print(f"Loaded trajectory with shape {last_traj.shape}, dtype {last_traj.dtype} on cpu")

In [ ]:
nomrd_vids = _normalize_elements_for_logging(
    last_traj,
    ["image min-max", "video min-max", "[-1;1] raw", "[-1;1] clipped"],
)

In [ ]:
gen_traj_index = 2  # what sample to visualize (sample = video)

In [ ]:
for norm_method, vid in nomrd_vids.items():
    print(
        f"Norm method: {norm_method} | saved data shape: {vid.shape} | selected video sample shape: {vid[gen_traj_index].shape}"
    )
    # Convert np array to video
    video_save_path = Path(output_dir, f"{norm_method}.mp4")
    imageio.mimwrite(video_save_path, vid[gen_traj_index].transpose(0, 2, 3, 1), fps=2)

    # Display the video with a specific width
    display(HTML(f'<video width="400" controls><source src="{video_save_path}" type="video/mp4"></video>'))